# Project 02 - No.8
## 網路股票討論度與波動度關係之探討

In [32]:
import requests
import time
from bs4 import BeautifulSoup
import os
import re
import urllib.request
import json
import jieba
import jieba.analyse
from pandas.core.frame import DataFrame

In [2]:
PTT_URL = 'https://www.ptt.cc'

In [3]:
def get_web_page(url):
    time.sleep(0.5)  # 每次爬取前暫停 0.5 秒以免被 PTT 網站判定為大量惡意爬取
    resp = requests.get(url=url)
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

In [51]:
def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html.parser')

    # 取得上一頁的連結
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        if d.find('div', 'date').string.strip() == date:  # 發文日期正確
            # 取得推文數
            push_count = 0
            if d.find('div', 'nrec').string:
                try:
                    push_count = int(d.find('div', 'nrec').string)  # 轉換字串為數字
                except ValueError:  # 若轉換失敗，不做任何事，push_count 保持為 0
                    pass

            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').string
                articles.append(
                    title,
                )
    return articles, prev_url


In [57]:
def parse(dom):
    soup = BeautifulSoup(dom, 'html.parser')
    links = soup.find(id='main-content').find_all('a')
    img_urls = []
    for link in links:
        if re.match(r'^https?://(i.)?(m.)?imgur.com', link['href']):
            img_urls.append(link['href'])
    return img_urls


In [251]:
if __name__ == '__main__':
    current_page = get_web_page(PTT_URL + '/bbs/Stock/index3846.html')
    if current_page:
        articles = []  # 全部的今日文章
        date = time.strftime("12/05").lstrip('0') # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        current_articles, prev_url = get_articles(current_page, "12/05")  # 目前頁面的今日文章
        while current_articles:  # 若目前頁面有今日文章則加入 articles，並回到上一頁繼續尋找是否有今日文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, "12/05")
            
        # 已取得文章列表，開始進入各文章讀圖
        
        for article in articles:
            print(article)
            

[標的] 緯創 3231 長多
[新聞] 榮成(1909)11月營收40億2479萬元
[新聞] 明年台股大盤 上看12,682點
[請益] 3006晶豪科
Re: [新聞] 東亞危機就在眼前！國泰航班空中目擊
[標的] 4736 泰博
Re: [標的] 台指期  多　缺口補完再空
[新聞] 致茂11月營運旺 營收衝歷史新高
[新聞]康控-KY出貨美特　公司：議價還在談
[請益] 投注一股最大比例應該為多少
[新聞] 當沖降稅上路半年 投資人低買高賣居多
Re: [請益] 股票不是賺就是賠 ...
[新聞] 陸資違法炒大同股票 金管會二度懲處 
Re: [標的] 2233宇隆
[其他] 最有感的金管會主委
[其他] 106年12月05日信用交易統計
[新聞] 中信金幫辜仲諒付保釋金 遭重罰千萬
Re: [請益] 晶電2448是不是差不多了
Re: [標的] 1906寶隆 1909榮成
[請益] 開信用戶進度
[新聞] 台灣仍輸！南韓明年經濟估成長3％ 人均
[請益] 這兩本書推薦嗎
[標的] Re: [標的] 3443 創意 空
[請益] 為什麼老一輩的人說到郭劉母女都超生氣
[請益] 有沒有值得珍藏的書籍
Re: [標的] 亞光 3019 多
[標的] 力麗 1444 多
[標的] 台指期 多
[標的] 9941 裕融
[其他] 十一月營收
[其他] 106年12月05日三大法人買賣金額統計表
[請益]6153套房中...
[請益] 國際不穩甚至打起來的狀況
[新聞] 1,000元定期定額台股基金好甜　備受投資
[新聞] 郭台銘：2018年經濟會比今年好
[標的] 6182合晶
[請益] 晶電2448是不是差不多了
[請益] 股票不是賺就是賠 ...
[新聞] 大立光11月營收登2年新高，12月估下滑
Re: [標的] 6180 橘子 (空)
Re: [請益] 股票不是賺就是賠 ...
[請益] 4147 中裕
Re: [新聞] 3D感測器題材夯，全新、穩懋與宏捷科連番
[新聞] 金管會開鍘　辜仲諒1億交保總座吳一揆遭
[標的] 凌巨 8105 請益
[新聞] 購併大鱷再現！三星未來準備在三大領域
[新聞] 金管會下周公布 金融業公司治理不佳名單
[新聞] BDI創四年新高 續漲空間大
[其他] 亞馬遜將語音服務帶入企業市場推出企業
[新聞] 郭台銘：十年後鴻海製

In [252]:
print (articles)

['[標的] 緯創 3231 長多', '[新聞] 榮成(1909)11月營收40億2479萬元', '[新聞] 明年台股大盤 上看12,682點', '[請益] 3006晶豪科', 'Re: [新聞] 東亞危機就在眼前！國泰航班空中目擊', '[標的] 4736 泰博', 'Re: [標的] 台指期  多\u3000缺口補完再空', '[新聞] 致茂11月營運旺 營收衝歷史新高', '[新聞]康控-KY出貨美特\u3000公司：議價還在談', '[請益] 投注一股最大比例應該為多少', '[新聞] 當沖降稅上路半年 投資人低買高賣居多', 'Re: [請益] 股票不是賺就是賠 ...', '[新聞] 陸資違法炒大同股票 金管會二度懲處 ', 'Re: [標的] 2233宇隆', '[其他] 最有感的金管會主委', '[其他] 106年12月05日信用交易統計', '[新聞] 中信金幫辜仲諒付保釋金 遭重罰千萬', 'Re: [請益] 晶電2448是不是差不多了', 'Re: [標的] 1906寶隆 1909榮成', '[請益] 開信用戶進度', '[新聞] 台灣仍輸！南韓明年經濟估成長3％ 人均', '[請益] 這兩本書推薦嗎', '[標的] Re: [標的] 3443 創意 空', '[請益] 為什麼老一輩的人說到郭劉母女都超生氣', '[請益] 有沒有值得珍藏的書籍', 'Re: [標的] 亞光 3019 多', '[標的] 力麗 1444 多', '[標的] 台指期 多', '[標的] 9941 裕融', '[其他] 十一月營收', '[其他] 106年12月05日三大法人買賣金額統計表', '[請益]6153套房中...', '[請益] 國際不穩甚至打起來的狀況', '[新聞] 1,000元定期定額台股基金好甜\u3000備受投資', '[新聞] 郭台銘：2018年經濟會比今年好', '[標的] 6182合晶', '[請益] 晶電2448是不是差不多了', '[請益] 股票不是賺就是賠 ...', '[新聞] 大立光11月營收登2年新高，12月估下滑', 'Re: [標的] 6180 橘子 (空)', 'Re: [請益] 股票不是賺就是賠 ...', '[請益] 4147 中裕', 'Re: [新聞] 3D感測器題材夯，全新、穩懋與宏捷科連番', '[新聞

In [253]:
keywords=jieba.analyse.extract_tags(str(articles),topK=50)

In [254]:
print("/".join(keywords))

新聞/請益/Re/12/u3000/台股/.../金管會/股票/11/營收/05/東西/其他/套牢/榮成/1909/東亞/危機/國泰/目擊/指期/106/辜仲諒/晶電/2448/台灣/亞光/3019/郭台銘/6182/合晶/3D/感測/器題/材夯/穩懋/宏捷/科連番/企業/設計/高點/景碩/多少/新高/到底/航班/操作/三星/明年


In [29]:
readTxtAll = open('title.txt','r').read() #第二個參數變成r，且是用read()的方式來從頭讀到尾
print(readTxtAll)


  7	Re: [請益] 技術分析上的矛盾                       12/30	shotholisi
  7	Re: [新聞] 散戶為什麼容易虧錢？                   12/30	ll6a
 X1	[新聞] 《食品股》黑心油獲賠1.2億元，味全走強      12/30	anti87
  3	Re: [標的] 3105穩懋。空                           12/30	dildochen
  8	[標的] 3479 安勤                                  12/30	AZD9291
 11	[請益] K線                                        12/30	j2708180
 46	[請益] 內線會跟嗎？                               12/30	tensionship
 98	[新聞] 基本工資3萬 中小企業︰會倒一半             12/30	cjol
  5	[心得] 2017投資回顧                               12/30	kyoking
  8	[請益] 分割虧損事業群                             12/30	birdyman
   	[新聞] 內地明年起 限制銀行卡在境外大額提款        12/30	schooldance
  爆	[心得] 2017 ETF費用整理                           12/30	meya
 11	Re: [請益] 技術分析上的矛盾                       12/31	allenhsiao
 X1	Fw: [新聞] 為了學費求包養！女留學生：找乾爹有錯   12/31	Schaffer
 39	[新聞]  宏碁黑馬總裁－陳俊聖                      12/31	ericliou
 19	Re: [新聞] 散戶為什麼容易虧錢？                   12/31	m4vu0
 18	[心得]  新手會遇到的問題—認知偏誤                 12/31	padbear
 20	[新聞] 出清美光持股　南亞科201

In [33]:
articles=DataFrame(readTxtAll)

ValueError: DataFrame constructor not properly called!

In [30]:
def get_keywords(content):
    keywords=jieba.analyse.extract_tags(str(content,topK=15))
    print("/".join(keywords))

In [ ]:
group=title.groupby('date')